In [43]:
import requests
import random

TOKEN = "pk_18e2ba4b80cb471aa96ba58fadec6b05" # edit if using your own IEX account
SYMBOLS = """abde,amd,googl,goog,adi,aapl,amat,asml,adsk,bidu,
    avgo,cdns,cern,chkp,csco,ctxs,fb,intc,intu,klac,
    lrcx,mxim,mchp,mu,msft,ntap,ntes,nvda,nxpi,qcom,
    swks,symc,snps,txn,vrsn,wdc,wday,xlnx"""
TYPES = "quote"

query = {"token": TOKEN, "symbols": SYMBOLS, "types": TYPES}
r = requests.get("https://cloud.iexapis.com/beta/stock/market/batch", params=query)
dict = r.json()

var_dict = {}
for key in dict:
    var_dict[key] = {"latestPrice": dict[key]["quote"]["latestPrice"], "latestVolume": dict[key]["quote"]["latestVolume"],
                    "marketCap": dict[key]["quote"]["marketCap"], "week52High": dict[key]["quote"]["week52High"],
                    "week52Low": dict[key]["quote"]["week52Low"], "peRatio": dict[key]["quote"]["peRatio"]}
    
estimatedPrices = {}
for company in var_dict:
    estimatedPrices[company] = random.random() * var_dict[company]["latestPrice"]
    + random.random() * var_dict[company]["latestVolume"]
    + random.random() * var_dict[company]["marketCap"]
    + random.random() * var_dict[company]["week52High"]
    + random.random() * var_dict[company]["week52Low"]
    + random.random() * var_dict[company]["peRatio"]

In [46]:
  print(estimatedPrices)

{'AMD': 15.36383570111675, 'GOOGL': 574.6344093428089, 'GOOG': 1140.2834840502762, 'ADI': 90.00803647344009, 'AAPL': 125.3712047408982, 'AMAT': 12.433946065907005, 'ASML': 128.43396505833908, 'ADSK': 21.30280320605947, 'BIDU': 93.0444963706757, 'AVGO': 153.08402147065905, 'CDNS': 14.640765277769603, 'CERN': 7.218017614417411, 'CHKP': 50.17924575610996, 'CSCO': 22.443317590434752, 'CTXS': 66.86943191064027, 'FB': 47.5344054684495, 'INTC': 3.020682410644192, 'INTU': 203.71229603579403, 'KLAC': 92.21918235189852, 'LRCX': 15.157874787300816, 'MXIM': 5.041917297038374, 'MCHP': 83.5669981138582, 'MU': 29.717135662276533, 'MSFT': 85.89887814166597, 'NTAP': 15.709797308010508, 'NTES': 42.461278297666816, 'NVDA': 129.2933396591308, 'NXPI': 86.34506070188006, 'QCOM': 33.31597650984758, 'SWKS': 69.21216742327316, 'SYMC': 3.4748186690248284, 'SNPS': 109.54812549361557, 'TXN': 51.583938226569735, 'VRSN': 22.168020302510882, 'WDC': 24.58174128028005, 'WDAY': 64.370323099042, 'XLNX': 39.6532969131626